# Exercise 1: Introduction to xarray


## Aim: Learn about what xarray is and how to create and look at a `DataArray`.

### Issues Covered:
- Importing `xarray`
- Loading a dataset using `xr.open_dataset()`
- Creating a `DataArray`
- Indexing, using `.loc()`, `.isel()` and `.sel()`

## 1. Introduction to multidimensional arrays

- Unlabelled N dimensional arrays of numbers are the most widely used data structure in scientific computing
- These arrays lack meaningful metadata so users must track indices in an arbitrary fashion

<img src="../images/multidimensional_array.png" width="800"/>

Can you think of any reasons why xarray might be preferred to pandas when working with multi-dimensional data like climate models?
(Hint: how many dimensions does a pandas dataframe have?)

In [1]:
# xarray is designed to handle data with multiple dimensions
# pandas is defined for 1D (series) and 2D (dataframe) structures.
# xarray allows you to work with labelled dimensions and coorfinates.
# pandas only offers labels through 'MultiIndex'
# xarray is built on netcdf model and understands CF conventions
# panfas doesn't natively support netcdf or cf conventions
# xarray supports metadata attached to datasets
# pandas metadata support is minimal

## 2. xarray architecture

1. Open the `'../data/tas_rcp45_2055_mon_avg_change.nc'` dataset and load it into an xarray `Dataset` called `ds`.
(Hint: Don't forget to import any packages you need)

In [2]:
import xarray as xr
ds = xr.open_dataset('../data/tas_rcp45_2055_mon_avg_change.nc')

2. Look at the parameters of the dataset.

In [3]:
ds

<xarray.Dataset> Size: 3MB
Dimensions:  (lon: 360, lat: 180, time: 12)
Coordinates:
  * lon      (lon) float64 3kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat      (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time     (time) object 96B 2065-01-30 12:00:00 ... 2065-12-30 12:00:00
Data variables:
    tas      (time, lat, lon) float32 3MB ...
Attributes: (12/31)
    CDI:                       Climate Data Interface version 1.5.6 (http://c...
    Conventions:               CF-1.4
    history:                   Sat Mar 07 00:12:23 2015: cdo setgridtype,lonl...
    institution:               Met Office Hadley Centre, Fitzroy Road, Exeter...
    institute_id:              MOHC
    experiment_id:             rcp45
    ...                        ...
    modeling_realm:            atmos
    realization:               1
    cmor_version:              2.4.0
    nco_openmp_thread_number:  1
    comment:                   CDO Statistics mean calculated at NCAS BADC : ...
    CDO:                       Climate Data Operators version 1.5.6.1 (http:/...

3. What are the dimensions and variables in this dataset? What does each represent? 

In [4]:
# The dimensions are `time`, `lat` and `lon`. 
# The primary variable is `tas` representing the temperature anomaly across these dimensions.

4. Find the name of the Data Variable, and use it to extract a `DataArray` called `temperature`.

In [5]:
# tas = temperature anomomaly
temperature = ds["tas"]

5. Take a look at the `temperature` data array and inspect its dimensions, coordinates and attributes. What are the specific dimensions and coordinates associated with it? What metadata (attributes) is provided?

In [6]:
# It is a 3D array with dimensions (time, lat, lon)
temperature

<xarray.DataArray 'tas' (time: 12, lat: 180, lon: 360)> Size: 3MB
[777600 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 3kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat      (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time     (time) object 96B 2065-01-30 12:00:00 ... 2065-12-30 12:00:00
Attributes:
    standard_name:     air_temperature
    long_name:         Near-Surface Air Temperature
    units:             K
    comment:           daily-mean near-surface (usually, 2 meter) air tempera...
    original_name:     mo: m01s03i236
    cell_methods:      time: mean
    history:           2010-10-29T11:35:40Z altered by CMOR: Treated scalar d...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

## 3. Label-based indexing

6. Select a subset of the `temperature` array using label-based indexing to get data at the position [0,0,0].

In [7]:
temperature[0,0,0]

<xarray.DataArray 'tas' ()> Size: 4B
[1 values with dtype=float32]
Coordinates:
    lon      float64 8B 0.0
    lat      float64 8B -89.5
    time     object 8B 2065-01-30 12:00:00
Attributes:
    standard_name:     air_temperature
    long_name:         Near-Surface Air Temperature
    units:             K
    comment:           daily-mean near-surface (usually, 2 meter) air tempera...
    original_name:     mo: m01s03i236
    cell_methods:      time: mean
    history:           2010-10-29T11:35:40Z altered by CMOR: Treated scalar d...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

7. Use `.loc` to find the lat and lon values at the time `2065-01-30`. 

In [8]:
temperature.loc['2065-01-30', :, :]

<xarray.DataArray 'tas' (time: 1, lat: 180, lon: 360)> Size: 259kB
[64800 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 3kB 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * lat      (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time     (time) object 8B 2065-01-30 12:00:00
Attributes:
    standard_name:     air_temperature
    long_name:         Near-Surface Air Temperature
    units:             K
    comment:           daily-mean near-surface (usually, 2 meter) air tempera...
    original_name:     mo: m01s03i236
    cell_methods:      time: mean
    history:           2010-10-29T11:35:40Z altered by CMOR: Treated scalar d...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

8. It's not ideal to have to keep track of which dimension is in which position. Instead, use `.isel` to use the dimension names to get the data in the first time, lat and lon position.

In [9]:
temperature.isel(time=0, lat=0, lon=0)

<xarray.DataArray 'tas' ()> Size: 4B
[1 values with dtype=float32]
Coordinates:
    lon      float64 8B 0.0
    lat      float64 8B -89.5
    time     object 8B 2065-01-30 12:00:00
Attributes:
    standard_name:     air_temperature
    long_name:         Near-Surface Air Temperature
    units:             K
    comment:           daily-mean near-surface (usually, 2 meter) air tempera...
    original_name:     mo: m01s03i236
    cell_methods:      time: mean
    history:           2010-10-29T11:35:40Z altered by CMOR: Treated scalar d...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

9. The previous method is still referring to a selection by integer position. Use `.sel` to give a labelled index with the named dimension to find the data at `time=2065-12-30`, `lat=-78.5`, `lon=11.0`.

In [10]:
temperature.sel(time='2065-12-30', lat=-78.5, lon=11.0)

<xarray.DataArray 'tas' (time: 1)> Size: 4B
[1 values with dtype=float32]
Coordinates:
    lon      float64 8B 11.0
    lat      float64 8B -78.5
  * time     (time) object 8B 2065-12-30 12:00:00
Attributes:
    standard_name:     air_temperature
    long_name:         Near-Surface Air Temperature
    units:             K
    comment:           daily-mean near-surface (usually, 2 meter) air tempera...
    original_name:     mo: m01s03i236
    cell_methods:      time: mean
    history:           2010-10-29T11:35:40Z altered by CMOR: Treated scalar d...
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...